In [1]:
import sys

sys.path.append("..")
%load_ext autoreload
%autoreload 2
from model.reasoning import ConnectionsDataModule
from trl_wrapper.trainer_wrapper import TrainerWrapper, GRPO_MATH_CONFIG, GRPO_CONNECTIONS_CONFIG
from trl_wrapper.wrapper_config import DatasetConfig
from IPython.display import display

wrapper = TrainerWrapper(GRPO_CONNECTIONS_CONFIG)

wrapper.init_data_module()

/Users/brianfitzgerald/Documents/GitHub/smolmodels/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-27 21:08:32,464	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-03-27 21:08:32.857 | INFO     | trl_wrapper.trainer_wrapper:__init__:264 - Setting padding side to: right
2025-03-27 21:08:32.858 | INFO     | trl_wrapper.trainer_wrapper:init_data_module:315 - Using chat template override: llama3
2025-03-27 21:08:32.858 | INFO     | trl_wrapper.wrapper_config:__init__:133 - Cache dir: ../dataset_caches/connections_data_module
Map: 100%|██████████| 353/353 [00:00<00:00, 14565.70 examples/s]


In [2]:
# sample = data_module.train_dataset[0]
# sample
wrapper.data_module.train_dataset
first_sample = wrapper.data_module.train_dataset[0]

conv = first_sample['prompt']
fake_thoughts = "<reasoning>\nFirst four are academic\n</reasoning>\n"
fake_response = {'role': "assistant", 'content': fake_thoughts + first_sample['answer'] + "\n"}
# conv = [*conv, {'role': "assistant", 'content': first_sample['answer']}]

In [4]:
# print(fake_response['content'])
display(first_sample['answer_formatted'])

[['dope', 'scoop', 'skinny', 'word'],
 ['con', 'dupe', 'fool', 'trick'],
 ['cant', 'lean', 'list', 'slope'],
 ['boob', 'eggshell', 'giggle', 'hello']]

In [6]:
for reward_func in wrapper.data_module.reward_functions():
    rew = reward_func([conv], [[fake_response]], answer=first_sample['answer_formatted'])

2025-03-27 21:10:04.022 | INFO     | model.reasoning:xmlcount_reward_func:61 - XML count rewards: [0.25]
2025-03-27 21:10:04.023 | INFO     | model.reasoning:strict_format_reward_func:107 - Strict format rewards: [0.0]
2025-03-27 21:10:04.023 | INFO     | model.reasoning:connections_reward_func:175 - Model generations: ['<reasoning>\nFirst four are academic\n</reasoning>\n<answer><group>dope, scoop, skinny, word</group>\n<group>con, dupe, fool, trick</group>\n<group>cant, lean, list, slope</group>\n<group>boob, eggshell, giggle, hello</group></answer>\n']
2025-03-27 21:10:04.023 | INFO     | model.reasoning:connections_reward_func:177 - Groups: [[['dope', 'scoop', 'skinny', 'word'], ['con', 'dupe', 'fool', 'trick'], ['cant', 'lean', 'list', 'slope'], ['boob', 'eggshell', 'giggle', 'hello']]]
2025-03-27 21:10:04.023 | INFO     | model.reasoning:connections_reward_func:179 - Connections scores: [4.0]
2025-03-27 21:10:04.023 | INFO     | model.reasoning:group_size_reward_func:188 - Group 